# importaciones

In [10]:
import pandas as pd
from statsmodels.formula.api import ols
import statsmodels.api as sm
from scipy.stats import chi2_contingency

# Cargar los datos
data = pd.read_csv('flights.csv')

# Hipótesis 1:
H0: No hay diferencia significativa en los retrasos de salida (DepDelay) entre las diferentes aerolíneas (Carrier).

H1: Hay una diferencia significativa en los retrasos de salida (DepDelay) entre las diferentes aerolíneas (Carrier).

In [11]:
# ANOVA entre el retraso de salida (DepDelay) y las aerolíneas (Carrier)
modelo_anova_dep = ols('DepDelay ~ C(Carrier)', data=data).fit()
tabla_anova_dep = sm.stats.anova_lm(modelo_anova_dep, typ=2)

# Imprimir los resultados de ANOVA de manera más clara
print("ANOVA Resultados - Hipótesis 1 (DepDelay vs Carrier):")
print(tabla_anova_dep)

ANOVA Resultados - Hipótesis 1 (DepDelay vs Carrier):
                  sum_sq        df           F  PR(>F)
C(Carrier)  3.063989e+06      15.0  161.933829     0.0
Residual    3.430093e+08  271924.0         NaN     NaN


H0: No hay diferencia significativa en los retrasos de salida (DepDelay) entre las diferentes aerolíneas (Carrier).

Resultados de ANOVA:

F: 161.93
p-value (PR(>F)): 0.0 (esto indica que el valor p es extremadamente pequeño, prácticamente cero)
Interpretación:

El valor p es significativamente menor a 0.05, lo que indica que hay diferencias significativas en los retrasos de salida entre las diferentes aerolíneas.
Por lo tanto, rechazamos la hipótesis nula (H0) y concluimos que el retraso en la salida depende de la aerolínea.

# Hipótesis 2:
H0: No hay diferencia significativa en el retraso promedio de llegada (ArrDelay) entre los aeropuertos de origen (OriginAirportID).
H1: Hay una diferencia significativa en el retraso promedio de llegada (ArrDelay) entre los aeropuertos de origen (OriginAirportID).

Estadístico de prueba: t-Student para dos grupos de aeropuertos principales (o prueba ANOVA si son múltiples grupos)

In [12]:
# ANOVA entre el retraso de llegada (ArrDelay) y el ID del aeropuerto de origen (OriginAirportID)
modelo_anova = ols('ArrDelay ~ C(OriginAirportID)', data=data).fit()
tabla_anova = sm.stats.anova_lm(modelo_anova, typ=2)

# Imprimir los resultados de ANOVA de manera más clara
print("ANOVA Resultados - Hipótesis 2 (ArrDelay vs OriginAirportID):")
print(tabla_anova)

ANOVA Resultados - Hipótesis 2 (ArrDelay vs OriginAirportID):
                          sum_sq        df          F         PR(>F)
C(OriginAirportID)  1.782871e+06      69.0  17.754031  1.630862e-210
Residual            3.956718e+08  271870.0        NaN            NaN


H0: No hay diferencia significativa en el retraso promedio de llegada (ArrDelay) entre los aeropuertos de origen (OriginAirportID).

Resultados de ANOVA:

F: 17.75
p-value: 1.63e-210 (esto es un valor extremadamente pequeño, cercano a 0)
Interpretación:

Nuevamente, el valor p es mucho menor a 0.05, lo que indica que hay diferencias significativas en los retrasos de llegada entre los diferentes aeropuertos de origen.
Por lo tanto, rechazamos la hipótesis nula (H0) y concluimos que el retraso promedio de llegada varía significativamente según el aeropuerto de origen.

# Hipótesis 3:
H0: El porcentaje de vuelos cancelados (Cancelled) es independiente del mes del año (Month).
H1: El porcentaje de vuelos cancelados (Cancelled) depende del mes del año (Month).

Estadístico de prueba: Chi-Cuadrada

In [13]:
# Crear una tabla de contingencia entre Month y Cancelled
contingency_table = pd.crosstab(data['Month'], data['Cancelled'])

# Realizar la prueba Chi-Cuadrada
chi2, p, dof, expected = chi2_contingency(contingency_table)

# Imprimir los resultados de manera más clara
print("Chi-Cuadrada - Hipótesis 3 (Month vs Cancelled):")
print(f"Chi2: {chi2}")
print(f"p-value: {p}")
print(f"Grados de libertad: {dof}")
print("Frecuencias esperadas:\n", expected)

Chi-Cuadrada - Hipótesis 3 (Month vs Cancelled):
Chi2: 416.2613664622779
p-value: 8.908574264959884e-87
Grados de libertad: 6
Frecuencias esperadas:
 [[37898.21437082   410.78562918]
 [38781.63877326   420.36122674]
 [38860.78093697   421.21906303]
 [40410.97806869   438.02193131]
 [39532.50005148   428.49994852]
 [35813.80763404   388.19236596]
 [37726.08016474   408.91983526]]


H0: El porcentaje de vuelos cancelados (Cancelled) es independiente del mes del año (Month).

Resultados de Chi-Cuadrada:

Chi2: 416.26
p-value: 8.91e-87 (esto es un valor extremadamente pequeño, cercano a 0)
Interpretación:

El valor p es mucho menor a 0.05, lo que indica que hay una relación significativa entre el mes del año y las cancelaciones.
Por lo tanto, rechazamos la hipótesis nula (H0) y concluimos que el porcentaje de vuelos cancelados depende del mes del año.